In [1]:
import pandas as pd

In [3]:
df_rate = pd.read_csv('../data/play_rate.csv')

In [183]:
df = pd.read_csv('../data/play_ds.csv')

/home/zhengyzhou040820/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
df.head(5)

,uid,device,song_id,date,play_time,song_length
0,168550892,ar,23491655,2017-03-30,254,254.0
1,168549788,ip,295469,2017-03-30,16,242.0
2,168532580,ar,21762903,2017-03-30,67,137.0
3,168543026,ar,6623026,2017-03-30,0,0.0
4,168532580,ar,21762903,2017-03-30,67,137.0


In [38]:
def cleanPlayTime(row):
    if row['play_time'] == 0:
        return 0
    if '>' in row['play_time'] :
        # print s.split('>')
        return row['play_time'].split('>')[0]
    else:
        return row['play_time']

In [40]:
df['play_time'].fillna('0', inplace=True)

In [41]:
df['play_time_2'] = df.apply(cleanPlayTime,axis = 1)

In [50]:
def calculateTime(row):
    if int(row['song_length']) == 0:
        #print("processed")
        return 0
    else:
        #print("processed")
        return float(row['play_time_2']) / float(row['song_length'])

In [51]:
df[['play_time_2']] = df[['play_time_2']].astype(float)
df['play_time_2'].fillna('0', inplace=True)
df['song_length'].fillna('0', inplace=True)

In [52]:
df.iloc[[748]]

,uid,device,song_id,date,play_time,song_length,play_time_2
748,168547485,ar,298250,2017-03-30,11>}(223.214.39.194)TM,193,11.0


In [53]:
df['play_ratio'] = df.apply(calculateTime,axis = 1)

In [54]:
df.head(5)

,uid,device,song_id,date,play_time,song_length,play_time_2,play_ratio
0,168550892,ar,23491655,2017-03-30,254,254,254.0,1.000000
1,168549788,ip,295469,2017-03-30,16,242,16.0,0.066116
2,168532580,ar,21762903,2017-03-30,67,137,67.0,0.489051
3,168543026,ar,6623026,2017-03-30,0,0,0.0,0.000000
4,168532580,ar,21762903,2017-03-30,67,137,67.0,0.489051


In [55]:
df.to_csv('../data/play_ds_extention.csv')

In [56]:
df_rate = df.groupby(['uid','song_id'])
df_rate = df.drop(['device','date','play_time','song_length','play_time_2'], axis=1)
df_rate = df_rate.groupby(['uid', 'song_id']).agg({'play_ratio':'sum'})
df_rate = df_rate.reset_index()
df_rate.to_csv('../data/play_rate.csv')

In [57]:
df_rate.head(5)

,uid,song_id,play_ratio
0,12333,703693,0.821429
1,12333,708428,9.682796
2,12333,875447,8.418803
3,12333,5.11457e+06,7.802120
4,12333,5.23738e+06,8.000000


In [77]:
df_rate[df_rate['play_ratio'] >= 1].count()

uid            629707
song_id        629707
play_ratio     629707
user_id_map    629707
song_id_map    629707
dtype: int64

In [78]:
df_rate[df_rate['play_ratio'] < 1].count()

uid            1105842
song_id        1105842
play_ratio     1105842
user_id_map    1105842
song_id_map    1105842
dtype: int64

In [ ]:
# Due to we have too much data, we have to resample based on play_ratio
from sklearn.utils import resample
# Separate majority and minority classes
df_majority = df_rate[df_rate.play_ratio < 1]
df_minority = df_rate[df_rate.play_ratio >= 1]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples= ,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.balance.value_counts()
# 1    49
# 0    49
# Name: balance, dtype: int64

In [58]:
df_rate = df_rate.dropna()

In [59]:
df_rate.to_csv('../data/play_rate.csv')

In [138]:
df_rate = pd.read_csv('../data/play_rate.csv')

In [139]:
df_rate.head(5)

,Unnamed: 0,uid,song_id,play_ratio
0,0,12333,703693.0,0.821429
1,1,12333,708428.0,9.682796
2,2,12333,875447.0,8.418803
3,3,12333,5114569.0,7.802120
4,4,12333,5237384.0,8.000000


In [140]:
df_rate = df_rate[df_rate['song_id'] > 0]

In [141]:
df_rate.size

6888228

In [143]:
df_rate['freq'] = df_rate.groupby('song_id')['song_id'].transform('count')

In [153]:
df_rate = df_rate[df_rate['freq'] >= 10]

In [154]:
df_rate.size

6259000

In [155]:
len(df_rate.song_id.unique())

20814

In [156]:
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [157]:
# convert uid to a smaller list
user_id_lst = df_rate['uid'].unique() 
user_id_dict = {k: v for v, k in enumerate(user_id_lst)}

# convert song id to a smaller list
song_id_lst = df_rate['song_id'].unique() 
song_id_dict = {k: v for v, k in enumerate(song_id_lst)}

In [158]:
df_rate['user_id_map'] = df_rate.apply(lambda x: user_id_dict[x['uid']],axis = 1)
df_rate['song_id_map'] = df_rate.apply(lambda x: song_id_dict[x['song_id']],axis = 1) 

In [33]:
highest_user_id = df_rate.user_id_map.max() + 1
highest_song_id = df_rate.song_id_map.max() + 1
ratings_mat = sparse.lil_matrix((highest_user_id, highest_song_id)) 
ratings_mat

<51154x20815 sparse matrix of type '<type 'numpy.float64'>'
	with 0 stored elements in LInked List format>

In [196]:
from math import fabs
for _, row in df_rate.iterrows():
# Due to some data error, use absolute to correct negative play time
# subtract 1 from id's due to match 0 indexing 
    ratings_mat[row.user_id_map-1, row.song_id_map-1] = fabs(row.play_ratio)

In [197]:
# # item_item recommendation
# utility_mat = ratings_mat
# Item-Item Similarity Matrix
item_sim_mat = cosine_similarity(ratings_mat.T)

In [198]:
import numpy as np
least_to_most_sim_indexes = np.argsort(item_sim_mat, axis=1)
# Neighborhoods
neighborhood_size = 75
neighborhoods = least_to_most_sim_indexes[:, -neighborhood_size:]

In [199]:
neighborhoods.shape

(20815, 75)

In [200]:
from time import time
user_id = 12333

n_users = ratings_mat.shape[0] 
n_items = ratings_mat.shape[1]

start_time = time()
items_rated_by_this_user = ratings_mat[user_id].nonzero()[1] 
# Just initializing so we have somewhere to put rating preds 
out = np.zeros(n_items)
for item_to_rate in range(n_items):
    relevant_items = np.intersect1d(neighborhoods[item_to_rate], 
                                    items_rated_by_this_user,
                                    assume_unique=True) 
    # assume_unique speeds up intersection 
    out[item_to_rate] = ratings_mat[user_id, relevant_items] * item_sim_mat[item_to_rate, relevant_items] / item_sim_mat[item_to_rate, relevant_items].sum()

pred_ratings = np.nan_to_num(out)
print(pred_ratings)
print("Execution time: %f seconds" % (time()-start_time))

/home/zhengyzhou040820/.local/lib/python2.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in divide
  app.launch_new_instance()


[0. 0. 0. ... 0. 0. 0.]
Execution time: 4.699014 seconds


In [201]:
# Get final recommendation # Recommend n movies 
n = 10

# Get item indexes sorted by predicted rating 
item_index_sorted_by_pred_rating = list(np.argsort(pred_ratings))[::-1]

# Find items that have been rated by user 
items_rated_by_this_user = ratings_mat[user_id].nonzero()[1]

# We want to exclude the items that have been rated by user 
unrated_items_by_pred_rating = [item for item in item_index_sorted_by_pred_rating if item not in items_rated_by_this_user]

unrated_items_by_pred_rating[:n]

[822, 16918, 17354, 19109, 1869, 10823, 1240, 11491, 996, 17654]

In [202]:
# Get song information
df_song_info = pd.read_csv('../data/play/song_info.csv')

In [203]:
df_song_info.head(5)

,Unnamed: 0,song_id,song_type,song_name,singer
0,0,2.34917e+07,0.0,裂心,林志炫
1,1,298250,0.0,See You Again,Miley Cyrus
2,2,1.18814e+07,0.0,一梦浮生,颜淡妖妖
3,3,295469,0.0,凡人歌,高胜美
4,4,2.13934e+07,0.0,重生（Intro）,幼稚园杀手


In [204]:
df_song_info = df_song_info.drop(['Unnamed: 0'],axis=1)

In [205]:
unrated_items_by_pred_rating[0]

822

In [206]:
df_rate[df_rate['song_id_map'] == 1]['song_id'].unique()[0]

708428.0

In [207]:
df_rate.head(5)

,Unnamed: 0,uid,song_id,play_ratio,freq,user_id_map,song_id_map
0,0,12333,703693.0,0.821429,12,0,0
1,1,12333,708428.0,9.682796,13,0,1
2,2,12333,875447.0,8.418803,11,0,2
3,3,12333,5114569.0,7.802120,4092,0,3
4,4,12333,5237384.0,8.000000,6079,0,4


In [208]:
# Show the corresponding 
recommend_list = []
for i in range(10):
    song_map_id = unrated_items_by_pred_rating[i]
    song_id = df_rate[df_rate['song_id_map'] == song_map_id]['song_id'].unique()[0]
    recommend_list.append(df_song_info[df_song_info['song_id'] == song_id].iloc[0])
recommend_list

[song_id      6.46889e+06
 song_type              0
 song_name             演员
 singer               薛之谦
 Name: 478, dtype: object, song_id      1.44159e+07
 song_type              0
 song_name            我的猫
 singer              吉克隽逸
 Name: 49721, dtype: object, song_id               1.10045e+06
 song_type                       0
 song_name    杯水情歌-(电视剧《281封信》片尾曲)
 singer                         庞龙
 Name: 35150, dtype: object, song_id      9.87818e+06
 song_type              0
 song_name       Jumpshot
 singer             Dawin
 Name: 80683, dtype: object, song_id      6.71477e+06
 song_type              0
 song_name            七月上
 singer             Jam阿敬
 Name: 3065, dtype: object, song_id                      7.10302e+06
 song_type                              0
 song_name    Every Time I Hear That Song
 singer                     Blake Shelton
 Name: 2914, dtype: object, song_id      696542
 song_type         0
 song_name       蓝精灵
 singer         儿童歌曲
 Name: 7254, dtype: object, 

In [209]:
df_rate.to_csv('../data/play_rate.csv')